In [172]:
import pandas as pd
import numpy as np
import os
import random
import scipy as sp
import sklearn
import us
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import time
import sys
import moment
import pickle as pkl

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [13]:
disasters = pd.read_csv("SF Summer Invitational Datasets/DisasterDeclarationsSummaries.csv")

In [14]:
d2017 = disasters.loc[flight_traffic['fyDeclared'] == 2017]
d2017

,disasterNumber,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,state,declarationDate,fyDeclared,disasterType,incidentType,title,incidentBeginDate,incidentEndDate,disasterCloseOutDate,declaredCountyArea,placeCode,hash,lastRefresh
41792,3381,0,0,1,0,CA,2017-02-14T19:20:00.000Z,2017,EM,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,2017-02-07T18:14:00.000Z,2017-02-23T23:59:00.000Z,NaN,Butte (County),99007.0,aec2c6d69676fd0b238d60bb1990a35c,2018-02-09T14:40:15.951Z
41851,3383,0,0,1,0,VI,2017-09-05T19:10:00.000Z,2017,EM,Hurricane,HURRICANE IRMA,2017-09-05T09:00:00.000Z,2017-09-07T23:59:00.000Z,NaN,St. Croix (Island) (County-equivalent),99010.0,cb5f7a485b14e6302b935324ef37c5fd,2018-02-09T14:40:15.952Z
41852,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Beauregard (Parish),99011.0,4b026ba238049e1d0bac089c15a48838,2018-05-18T13:24:06.191Z
41853,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Sabine (Parish),99085.0,e1acd3a9108060a2f81c7431203baf57,2018-05-18T13:24:06.200Z
41854,3381,0,0,1,0,CA,2017-02-14T19:20:00.000Z,2017,EM,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,2017-02-07T18:14:00.000Z,2017-02-23T23:59:00.000Z,NaN,Yuba (County),99115.0,b971e65505060abe82473eb84f31aa62,2018-02-09T14:40:15.993Z
41855,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Natchitoches (Parish),99069.0,0948d18bf2a267c40f606d10be135313,2018-05-18T13:24:06.192Z
41856,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Allen (Parish),99003.0,1599d592aed0151ae586822a353ebd94,2018-05-18T13:24:06.176Z
41857,3381,0,0,1,0,CA,2017-02-14T19:20:00.000Z,2017,EM,Dam/Levee Break,POTENTIAL FAILURE OF THE EMERGENCY SPILLWAY AT...,2017-02-07T18:14:00.000Z,2017-02-23T23:59:00.000Z,NaN,Sutter (County),99101.0,fcdbf794dc0357ffb92af4825602135e,2018-02-09T14:40:15.940Z
41858,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Jefferson Davis (Parish),99053.0,45e9990b464e464c583e6f526888fcb6,2018-05-18T13:24:06.182Z
41859,3382,0,0,1,0,LA,2017-08-28T08:01:00.000Z,2017,EM,Hurricane,TROPICAL STORM HARVEY,2017-08-27T08:09:00.000Z,2017-09-10T23:59:00.000Z,2018-05-17T23:59:00.000Z,Cameron (Parish),99023.0,7ff4aba66293a1ddf27b843b9e313245,2018-05-18T13:24:06.193Z


In [15]:
d2017.describe(include=np.object)

,state,declarationDate,disasterType,incidentType,title,incidentBeginDate,incidentEndDate,disasterCloseOutDate,declaredCountyArea,hash,lastRefresh
count,1741,1741,1741,1741,1741,1741,1707,320,1741,1741,1741
unique,39,130,3,8,96,120,85,5,812,1741,991
top,GA,2017-09-08T12:35:00.000Z,DR,Hurricane,HURRICANE IRMA,2017-09-07T09:00:00.000Z,2017-09-20T23:59:00.000Z,2017-12-18T23:59:00.000Z,Jefferson (County),14759a2332a4ac3aed1f70aa0ac91ea5,2018-02-09T14:40:27.846Z
freq,352,159,1056,1063,733,318,321,226,12,1,3


In [16]:
d2017['title'].value_counts()

HURRICANE IRMA                                                       733
HURRICANE MARIA                                                      162
SEVERE WINTER STORMS, FLOODING, AND MUDSLIDES                        115
SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS, AND FLOODING           93
SEVERE WINTER STORM                                                   59
SEVERE STORMS, TORNADOES, STRAIGHT-LINE WINDS AND FLOODING            55
HURRICANE HARVEY                                                      53
HURRICANE NATE                                                        48
TROPICAL STORM HARVEY                                                 32
SEVERE WINTER STORM AND SNOWSTORM                                     28
SEVERE STORMS, TORNADOES, AND STRAIGHT-LINE WINDS                     28
SEVERE WINTER STORM, SNOWSTORM, STRAIGHT-LINE WINDS, AND FLOODING     27
SEVERE STORMS AND FLOODING                                            27
SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSLIDES 

In [174]:
d2017.incidentType.unique()

array(['Dam/Levee Break', 'Hurricane', 'Flood', 'Severe Storm(s)',
       'Tornado', 'Fire', 'Severe Ice Storm', 'Snow'], dtype=object)

In [26]:
incident_types = d2017.incidentType.unique()
incidentMap = {}
for i in range(len(incident_types)):
    incidentMap[incident_types[i]] = i

In [168]:
d2017.incidentEndDate.unique()

array(['2017-02-23T23:59:00.000Z', '2017-09-07T23:59:00.000Z',
       '2017-09-10T23:59:00.000Z', '2017-09-13T23:59:00.000Z',
       '2017-10-18T23:59:00.000Z', '2017-09-20T23:59:00.000Z',
       '2017-09-14T23:59:00.000Z', '2017-10-04T23:59:00.000Z',
       '2017-11-15T23:59:00.000Z', '2017-09-22T23:59:00.000Z',
       '2017-10-08T23:59:00.000Z', '2017-10-10T23:59:00.000Z',
       '2017-10-11T23:59:00.000Z', '2016-09-24T23:59:00.000Z',
       '2016-10-21T14:27:00.000Z', '2017-01-02T23:59:00.000Z',
       '2017-01-21T00:00:00.000Z', '2016-10-15T23:59:00.000Z',
       '2017-01-22T23:59:00.000Z', '2016-12-09T00:01:00.000Z',
       '2016-12-17T23:59:00.000Z', '2016-12-26T23:59:00.000Z',
       '2017-02-07T23:59:00.000Z', '2017-01-12T23:59:00.000Z',
       '2017-01-16T10:54:00.000Z', '2017-01-14T23:59:00.000Z',
       '2017-01-16T06:00:00.000Z', '2017-01-23T23:59:00.000Z',
       '2017-01-05T23:00:00.000Z', '2017-02-22T21:45:00.000Z',
       '2017-02-22T23:59:00.000Z', '2017-02-27T23:59:00

In [17]:
disaster_FIPS = defaultdict(int)
for index, row in d2017.iterrows():
    full_fips = us.states.lookup(row["state"]).fips
    disaster_FIPS[us.states.lookup(row["state"]).fips + str(int(row["placeCode"]))[2:5]] += 1

svg = open('SF Summer Invitational Datasets/counties.svg', 'r').read()

# Load into Beautiful Soup
soup = BeautifulSoup(svg, 'xml')

# Find counties
paths = soup.findAll('path')

# Map colors
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]
 
# County style
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;\
stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;\
marker-start:none;stroke-linejoin:bevel;fill:'

# Color the counties based on unemployment rate
for p in paths:
     
    if p['id'] not in ["State_Lines", "separator"]:
        try:
            rate = disaster_FIPS[p['id']]
        except:
            continue
             
        if rate > 4:
            color_class = 5
        elif rate == 4:
            color_class = 4
        elif rate == 3:
            color_class = 3
        elif rate == 2:
            color_class = 2
        elif rate == 1:
            color_class = 1
        else:
            color_class = 0
 
        color = colors[color_class]
        p['style'] = path_style + color
        
print(soup.prettify(), file=open("disasters.svg", "w"))


In [48]:
airports = pd.read_csv("SF Summer Invitational Datasets/airports.csv", )
# IN THIS QUARTER, HOW MANY NATURAL DISASTERS FROM THE ORIGIN AIRPORT HAD OCCURRED
# airport_to_fips = {}
for index, row in airports.iterrows():
    if not row["airport_id"] in airport_to_fips and not pd.isna(row["latitude"]) and not pd.isna(row["longitude"]):
        r = requests.get('https://geo.fcc.gov/api/census/block/find?latitude=' + str(row["latitude"]) + '&longitude=' + str(row["longitude"]) + '&format=json')
        response = r.json()
        airport_to_fips[row["airport_id"]] = response['County']['FIPS']
        time.sleep(0.5)

In [59]:
airport_FIPS = defaultdict(int)
for index, row in airports.iterrows():
    if row["airport_id"] in airport_to_fips:
        full_fips = airport_to_fips[row["airport_id"]]
        airport_FIPS[full_fips] += 1

svg = open('SF Summer Invitational Datasets/counties.svg', 'r').read()

# Load into Beautiful Soup
soup = BeautifulSoup(svg, 'xml')

# Find counties
paths = soup.findAll('path')

# Map colors
colors = ["#EDFAFF", "#9BE6FF", "#59D5FF", "#14C4FF"]
 
# County style
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;\
stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;\
marker-start:none;stroke-linejoin:bevel;fill:'

# Color the counties based on unemployment rate
for p in paths:
     
    if p['id'] not in ["State_Lines", "separator"]:
        try:
            rate = airport_FIPS[p['id']]
        except:
            continue
             
        if rate >= 3:
            color_class = 3
        elif rate == 2:
            color_class = 2
        elif rate == 1:
            color_class = 1
        else:
            color_class = 0
 
        color = colors[color_class]
        p['style'] = path_style + color
        
print(soup.prettify(), file=open("airports.svg", "w"))


In [159]:
fares = pd.read_csv("SF Summer Invitational Datasets/fares.csv")
fares

,quarter,airline_id,origin_airport,destination_airport,distance,10,20,30,40,50,...,2410,2420,2430,2440,2450,2460,2470,2480,2490,2500
0,1,AA,CLT,SGF,708,2,6,11,15,32,...,0,0,0,0,0,0,0,0,0,0
1,1,DL,DTW,SAN,1956,13,4,3,2,8,...,0,0,5,0,0,0,0,0,0,28
2,1,DL,MSP,ISN,546,7,11,14,36,52,...,0,0,0,0,0,0,0,0,0,0
3,1,AA,CLT,RDU,130,2573,1665,927,556,356,...,0,0,0,0,0,0,0,0,0,0
4,1,DL,ATL,RIC,481,112,186,379,650,799,...,0,0,0,0,0,0,0,0,0,0
5,1,AA,CLT,TPA,507,114,225,527,822,835,...,0,0,0,0,0,0,0,0,0,2
6,1,AA,DFW,IND,761,23,39,33,61,130,...,0,0,0,0,0,0,0,0,0,0
7,1,UA,KOA,LAX,2504,15,0,0,0,1,...,0,0,0,2,0,0,0,0,0,17
8,1,AA,DFW,MHK,431,12,33,70,78,74,...,0,0,0,0,0,0,0,0,0,0
9,1,DL,LGA,MCO,950,7,7,24,38,242,...,0,0,0,0,0,0,0,0,0,2


In [170]:
# map: FIPS and quarter
disaster_fips_quarter = {}
for i, r in d2017.iterrows():
    incidentStartMonth = int(r["incidentBeginDate"][5:7])
    incidentStartQuarter = int((incidentMonth - 1) / 3) + 1
    incidentEndMonth = int(r["incidentEndDate"][5:7]) if not pd.isna(r["incidentEndDate"]) else incidentStartMonth
    incidentEndQuarter = int((incidentEndMonth - 1) / 3) + 1
    full_fips_incident = us.states.lookup(r["state"]).fips + str(int(r["placeCode"]))[2:5]
    incident_type_id = incidentMap[r["incidentType"]]
    full_id = str(incidentQuarter) + full_fips_incident
    full_end_id = str(incidentEndQuarter) + full_fips_incident
    if full_id not in disaster_fips_quarter:
        disaster_fips_quarter[full_id] = []
    disaster_fips_quarter[full_id].append(incident_type_id)
    if full_id != full_end_id:
        if full_end_id not in disaster_fips_quarter:
            disaster_fips_quarter[full_end_id] = []
        disaster_fips_quarter[full_end_id].append(incident_type_id)

In [171]:
numMatches = 0
fare_disasters = np.zeros((fares.shape[0], 8))
for index, row in fares.iterrows():
    if index % 5000 == 0:
        print(index)
        sys.stdout.flush()
    origin_airport = row["origin_airport"]
    if origin_airport in airport_to_fips and not pd.isna(row["quarter"]) and airport_to_fips[origin_airport] is not None:
        fips_and_quarter_id = str(row["quarter"]) + airport_to_fips[origin_airport]
        if fips_and_quarter_id in disaster_fips_quarter:
            numMatches += 1
            for disaster_type in disaster_fips_quarter[fips_and_quarter_id]:
#                 numMatches += 1
                fare_disasters[index][disaster_type] += 1

print(numMatches)
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
14794


In [181]:
hhi = pkl.load(open("HHI_route.pickle", "rb"))
type(hhi)
hhi

defaultdict(list,
            {('SLC', 'TPA'): [-1, -1, -1, 1],
             ('MSP', 'HNL'): [1, 1, 1, 1],
             ('MSP', 'LAN'): [1, 0.49519890260630994, 1, 1],
             ('FLL', 'DTW'): [0.2326609823869132,
              0.20525118839984946,
              0.18338468316919573,
              0.1511702207077884],
             ('OAK', 'MCI'): [1, 1, 1, 1],
             ('TUS', 'DEN'): [0.06048268316290031,
              0.08254564028397128,
              0.24395156733080348,
              0.15109619669729246],
             ('ATL', 'GPT'): [0.4049586776859506,
              0.6808888848477683,
              0.7011761864509114,
              0.7813297193877551],
             ('PIT', 'MSP'): [0.20847440238021941,
              0.24266057525951557,
              0.4168195794485221,
              0.30362349021241153],
             ('PHX', 'SMF'): [0.0017880816326529825,
              0.0011275514961017752,
              0.00013981464572676572,
              0.24785344175124682],
    

In [185]:
fips_to_competitiveness = {}
for key, val in hhi.items():
    if val[0] == -1:
        continue
    origin, dest = key
    if origin not in airport_to_fips:
        continue
    fips = airport_to_fips[origin]
    # use only q1
    if fips not in fips_to_competitiveness:
        fips_to_competitiveness[fips] = []
    fips_to_competitiveness[fips].append(val[0])
    
for key, val in fips_to_competitiveness.items():
    fips_to_competitiveness[key] = np.mean(np.array(val))
    
fips_to_competitiveness

{'27053': 0.67041745919076035,
 '12011': 0.55377213922701973,
 '06001': 0.81225416664260741,
 '04019': 0.74590462057336593,
 '13063': 0.58483342024661311,
 '42003': 0.67730980219429759,
 '04013': 0.60795553032391081,
 '12057': 0.72541591125751337,
 '48453': 0.67060764901663272,
 '41051': 0.64807543174361748,
 '06037': 0.59215254240083892,
 '26043': 1.0,
 '48201': 0.81467332430590822,
 '01097': 0.80526623576485068,
 '20173': 0.81568756540273735,
 '32003': 0.62527823546163996,
 '08031': 0.55917100826312482,
 '19153': 0.84150299474316148,
 '48439': 0.86372253936309829,
 '23005': 0.90451388888888884,
 '26163': 0.55130348137134366,
 '36081': 0.6773875950059246,
 '51013': 0.75768535942945081,
 '49035': 0.60753912596070125,
 '51107': 0.77422781189124545,
 '15001': 0.89685992653018387,
 '12099': 0.73162291176147365,
 '29165': 0.71345947296984513,
 '06081': 0.67072133331846484,
 '48113': 0.90097060377510152,
 '02282': 1.0,
 '39035': 0.58524632454258896,
 '05007': 0.82919575446398053,
 '06085': 

In [193]:
hhi_airport = pkl.load(open("HHI.pickle", "rb"))
fips_to_competitiveness = {}
for key, val in hhi_airport.items():
    if key not in airport_to_fips:
        continue
    fips = airport_to_fips[key]
    fips_to_competitiveness[fips] = np.mean(np.array(val))
    
fips_to_competitiveness

{'36081': 0.080756019140163809,
 '45083': 0.089254059289580884,
 '26049': 0.021340756648065173,
 '02105': 1.0,
 '02220': 0.79854433928435908,
 '12086': 0.50498466086922034,
 '25025': 0.14578715087256972,
 '15007': 0.39389546034825834,
 '36007': 1.0,
 '05119': 0.037657545729738597,
 '16001': 0.25831476738275461,
 '23019': 0.37339380551524909,
 '04027': 1.0,
 '08067': 0.90686142264359404,
 '21145': 1.0,
 '29019': 0.92031997984378933,
 '56025': 1.0,
 '13245': 0.18001317685020901,
 '08037': 0.36723449861824919,
 '16077': 1.0,
 '06073': 0.15743796105416033,
 '17143': 0.41341248754550852,
 '28067': 0.79475365461818681,
 '45051': 0.36409013983540184,
 '35049': 0.22656407063471845,
 '51013': 0.11319301035149945,
 '17167': 0.77054730988578291,
 '48303': 0.2381945056581134,
 '01089': 0.097041791229791508,
 '26103': 1.0,
 '46103': 0.83251964549850255,
 '24003': 0.48074262471284163,
 '18003': 0.76674238260960481,
 '51087': 0.089635141946346142,
 '48029': 0.20891488181363199,
 '02275': 1.0,
 '27137

In [194]:
svg = open('SF Summer Invitational Datasets/counties.svg', 'r').read()

# Load into Beautiful Soup
soup = BeautifulSoup(svg, 'xml')

# Find counties
paths = soup.findAll('path')

# County style
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;\
stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;\
marker-start:none;stroke-linejoin:bevel;fill:'

def fadeColor(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    assert len(c1)==len(c2)
    assert mix>=0 and mix<=1, 'mix='+str(mix)
    rgb1=np.array([int(c1[ii:ii+2],16) for ii in range(1,len(c1),2)])
    rgb2=np.array([int(c2[ii:ii+2],16) for ii in range(1,len(c2),2)])   
    rgb=((1-mix)*rgb1+mix*rgb2).astype(int)
    c='#'+''.join([hex(a)[2:] for a in rgb])
    return c

c1='#F2ED6F' #blue
c2='#587792' #green

# Color the counties based on unemployment rate
for p in paths:
     
    if p['id'] not in ["State_Lines", "separator"]:
        try:
            rate = fips_to_competitiveness[p['id']]
        except:
            continue
             
        if rate == -1:
            continue
        else:
            color = fadeColor(c1, c2, rate)
        p['style'] = path_style + color
        
print(soup.prettify(), file=open("hhi_airport.svg", "w"))


In [ ]:
X = []
X = np.concatenate(X)